In [1]:
import os
import cv2
import openslide
import numpy as np
import pandas as pd
import tensorflow as tf

from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from keras import layers, models
from keras import backend as K
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from openslide.deepzoom import DeepZoomGenerator

from preprocessing import create_patches

Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50

In [3]:
model = ResNet50(weights=None, 
                 include_top=True,
                 classes=2,
                 input_shape=(256, 256, 3))

In [4]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'])

In [5]:
model.load_weights('./model/resnet50.h5')

In [6]:
slide_path = '../data/train/image/positive/Slide003.mrxs'
truth_path = '../data/train/mask/positive/Slide003.png'

In [7]:
file_handles = []
def test_generator(samples,
                   slide_path,
                   batch_size,
                   patch_size=256,
                   shuffle=True):
    
    slide = openslide.open_slide(slide_path)
    file_handles.append(slide)

    # tiles
    tiles = DeepZoomGenerator(slide, tile_size=patch_size, overlap=0, limit_bounds=False)

    num_samples = len(samples)
    while 1:
        if shuffle:
            samples = samples.sample(frac=1)  # shuffling

        for offset in range(0, num_samples, batch_size):
            batch_samples = samples.iloc[offset:offset+batch_size]

            batch_tiles= []
            for slide_path, (y, x) in zip(batch_samples['slide_path'].values, 
                                          batch_samples['tile_loc'].values):
                img = tiles.get_tile(tiles.level_count-1, (x, y))             
                
                if img.size != (patch_size, patch_size):
                    img = Image.new('RGB', (patch_size, patch_size))
                    
                batch_tiles.append(np.array(img))
                
            # train_x
            test_x = np.array(batch_tiles)
            yield test_x

In [8]:
def get_test_path():
    slide_paths = {}
    files = os.listdir('/data/test/')
    for file_ in files:
        if 'mrxs' in file_:
            fname = file_.split('.')[0]
            slide_paths[fname] = '/data/test/' + file_
        
    return slide_paths

In [9]:
test_paths = {'Slide003': '../data/test/Slide003.mrxs'}

In [10]:
patch_size = 256
batch_size = 32

In [12]:
slide_path = '../data/test/Slide003.mrxs'

In [13]:
samples = create_patches(slide_path, 'test')

/home/excelsiorcjh/D/dev/help/notebook/wide-res-net/preprocessing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  samples['tile_loc'] = list(samples.index)


In [14]:
num_samples = len(samples)
if num_samples > 5000:
    num_samples = 5000

In [15]:
samples = samples.sample(num_samples, random_state=42)
samples.reset_index(drop=True, inplace=True)

In [16]:
test_gen = test_generator(samples, slide_path, batch_size)
test_steps = np.ceil(len(samples) / batch_size)

In [17]:
predicts = model.predict_generator(test_gen,
                                   steps=test_steps)

In [19]:
model.p

(5000, 2)